# Creating an Audioset subset

First, we need to download the full [Audioset](https://research.google.com/audioset/download.html) dataset. This is about 2.5GB so it may take some time to download.

### NOTE: The Audioset data is stored with filenames that are case sensitive. If you are using a filesystem with case-insensitive filenames (such as macOS) 75% of the dataset will be overwritten when you decompress the archive. You should only run this on a Linux machine.

In [16]:
import urllib.request

url = 'http://storage.googleapis.com/us_audioset/youtube_corpus/v1/features/features.tar.gz'
output_file = 'features.tar.gz'

urllib.request.urlretrieve(url, output_file)
print(f"✅ Tải thành công: {output_file}")


✅ Tải thành công: features.tar.gz


In [17]:
import urllib.request

url = 'http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/unbalanced_train_segments.csv'
output_file = 'unbalanced_train_segments.csv'

urllib.request.urlretrieve(url, output_file)
print(f"✅ Tải thành công: {output_file}")

✅ Tải thành công: unbalanced_train_segments.csv


In [ ]:
import os
import tarfile
import shutil

# ====== Cấu hình ======
FEATURES_TAR_PATH = "features.tar.gz"  # Đường dẫn file nén gốc
TEMP_EXTRACT_DIR = "features_tmp"      # Thư mục giải nén tạm
DEST_DIR = "audioset_v1_embeddings"    # Thư mục chứa kết quả
PARTS = {
    "audioset_v1_embeddings/eval": "eval",
    "audioset_v1_embeddings/unbal_train": "unbal_train",
    "audioset_v1_embeddings/bal_train": "bal_train"
}

# ====== Bước 1: Xoá thư mục đích nếu đã tồn tại ======
if os.path.exists(TEMP_EXTRACT_DIR):
    shutil.rmtree(TEMP_EXTRACT_DIR)
if os.path.exists(DEST_DIR):
    shutil.rmtree(DEST_DIR)

print("🧹 Đã xoá thư mục cũ (nếu có).")

# ====== Bước 2: Giải nén vào thư mục tạm ======
with tarfile.open(FEATURES_TAR_PATH, "r:gz") as tar:
    tar.extractall(TEMP_EXTRACT_DIR)

print("✅ Giải nén thành công vào thư mục tạm:", TEMP_EXTRACT_DIR)

# ====== Bước 3: Di chuyển từng phần vào thư mục đích ======
os.makedirs(DEST_DIR, exist_ok=True)

for src_name, dest_name in PARTS.items():
    src_path = os.path.join(TEMP_EXTRACT_DIR, src_name)
    dest_path = os.path.join(DEST_DIR, dest_name)
    if os.path.exists(src_path):
        shutil.move(src_path, dest_path)
        print(f"✅ Đã di chuyển: {src_name} ➜ {dest_path}")
    else:
        print(f"⚠️ Không tìm thấy {src_name}, có thể thiếu trong file nén.")

# ====== Bước 4: Kiểm tra kết quả ======
for _, folder in PARTS.items():
    final_path = os.path.join(DEST_DIR, folder)
    if os.path.exists(final_path):
        count = len(os.listdir(final_path))
        print(f"📁 {folder}: {count} file(s)")
    else:
        print(f"❌ {folder} không tồn tại.")

print("🎉 Hoàn tất quá trình giải nén và tổ chức thư mục.")


🧹 Đã xoá thư mục cũ (nếu có).
✅ Giải nén thành công vào thư mục tạm: features_tmp
⚠️ Không tìm thấy eval, có thể thiếu trong file nén.
⚠️ Không tìm thấy unbalanced_train, có thể thiếu trong file nén.
⚠️ Không tìm thấy balanced_train, có thể thiếu trong file nén.
❌ eval không tồn tại.
❌ unbal_train không tồn tại.
❌ bal_train không tồn tại.
🎉 Hoàn tất quá trình giải nén và tổ chức thư mục.


In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import glob


In order to run this on a linux machine, I used this notebook in [Google Colaboratory](https://colab.research.google.com/)
The following cells are for the uploading of necessary files to the colab instance.
If you are running this locally, you can skip this section

In [15]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
for k,v in uploaded.items():
  with open(k,'wb') as f:
    f.write(v)

In [ ]:
uploaded.keys()

Here is where the laughter and not laughter labels are loaded from local .csv files.
If you want to create a subset using a different category of labels, create new files containing a list of the labels you want to select as your positive and negative classes. You can find a list of all the Audioset labels [here](class_labels_indices.csv)

In [21]:
labels = pd.read_csv('unbalanced_train_segments.csv',header=2, quotechar=r'"',skipinitialspace=True)
laugh_labels = pd.read_csv('laugh_labels.csv',names=['num','label','description'])
not_laugh_labels = pd.read_csv('human_non_laugh_labels.csv',names=['num','label','description'])
l_str = '|'.join(laugh_labels['label'].values)

In [22]:
n_str = '|'.join(not_laugh_labels['label'].values)
labels['not_laughter'] = (labels['positive_labels'].str.contains(n_str) & ~labels['positive_labels'].str.contains(l_str))

## Eval set

In [26]:
%%time
import pandas as pd
import tensorflow as tf
import glob

# ===== BƯỚC 1: Định nghĩa nhãn tiếng cười và không cười =====
laugh_labels = pd.read_csv('laugh_labels.csv', names=['num','label','description'])
not_laugh_labels = pd.read_csv('human_non_laugh_labels.csv', names=['num','label','description'])

l_str = '|'.join(laugh_labels['label'].values)
n_str = '|'.join(not_laugh_labels['label'].values)

# ===== BƯỚC 2: Đọc eval_segments và tạo nhãn =====
labels = pd.read_csv('eval_segments.csv', header=2, quotechar='"', skipinitialspace=True)
labels['laughter'] = labels['positive_labels'].str.contains(l_str)
labels['not_laughter'] = (labels['positive_labels'].str.contains(n_str) & ~labels['positive_labels'].str.contains(l_str))

# ===== BƯỚC 3: Lấy mẫu dữ liệu dương (có tiếng cười) và âm =====
positive = labels[labels['laughter'] == True]
negative = labels[labels['not_laughter'] == True].sample(positive.shape[0], random_state=42)

# ===== BƯỚC 4: Gộp lại thành tập huấn luyện và lưu file CSV =====
subset = pd.concat([positive, negative], ignore_index=True)
subset.to_csv('eval_laugh_speech_training_subset.csv', index=False)
print(f"Tổng số mẫu: {subset.shape[0]}")

# ===== BƯỚC 5: Lọc TFRecord chứa các đoạn trong subset =====
files = glob.glob('audioset_v1_embeddings/eval/*')
subset_ids = set(subset['# YTID'].values)

i = 0
writer = tf.io.TFRecordWriter('eval_laugh_speech_subset.tfrecord')

for tfrecord in files:
    for example in tf.compat.v1.io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode('utf-8')
        if vid_id in subset_ids:
            writer.write(example)
            i += 1

writer.close()
print(f"✅ Số TFRecord đã ghi: {i}")


Tổng số mẫu: 586
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
✅ Số TFRecord đã ghi: 209
CPU times: total: 1.97 s
Wall time: 4.46 s


## train set

In [27]:
!pip install tqdm
from tqdm import tqdm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Warning: The audioset dataset is large and this will take a while to run. It took about 2 hours to process.

In [29]:
%%time
import pandas as pd
import tensorflow as tf
import glob
from tqdm import tqdm

# Đọc file nhãn
labels = pd.read_csv('unbalanced_train_segments.csv', header=2, quotechar='"', skipinitialspace=True)

# Giả sử bạn đã có 2 DataFrame: laugh_labels và not_laugh_labels
l_str = '|'.join(laugh_labels['label'].values)
n_str = '|'.join(not_laugh_labels['label'].values)

# Gán nhãn cười và không cười
labels['laughter'] = labels['positive_labels'].str.contains(l_str)
labels['not_laughter'] = (labels['positive_labels'].str.contains(n_str) & ~labels['positive_labels'].str.contains(l_str))

# Lấy mẫu
positive = labels[labels['laughter'] == True]
negative = labels[labels['not_laughter'] == True].sample(positive.shape[0], random_state=42)

# Gộp 2 nhóm và lưu
subset = pd.concat([positive, negative], ignore_index=True)
subset.to_csv('laugh_speech_training_subset.csv', index=False)
print(subset.shape[0])

# Lọc TFRecord
files = glob.glob('audioset_v1_embeddings/unbal_train/*')
subset_ids = set(subset['# YTID'].values)

i = 0
writer = tf.io.TFRecordWriter('bal_laugh_speech_subset.tfrecord')

for tfrecord in tqdm(files):
    for example in tf.compat.v1.io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode('utf-8')
        if vid_id in subset_ids:
            writer.write(example)
            i += 1

writer.close()
print(f"✅ Tổng TFRecord ghi: {i}")


18768


100%|██████████| 1444/1444 [00:06<00:00, 226.60it/s]


✅ Tổng TFRecord ghi: 6900
CPU times: total: 2.92 s
Wall time: 9.31 s


This is only necessary if you want to download from Colab

In [30]:
from google.colab import files
files.download('bal_laugh_speech_subset.tfrecord')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
files.download('eval_laugh_speech_subset.tfrecord')